## Tobigs 8주차 Reinforcement Learning
#### 15기 이윤정
---   

## `[Playing Atari with Deep Reinforcement Learning] Paper Review`
* 논문 저자 : Volodymyr Mnih (DeepMind)
* 논문 링크 : https://arxiv.org/pdf/1312.5602.pdf
---
   ## 1. Abstract
   * **최초로 Reinforcement Learning (이하 RL)을 통해 학습의 성공한 Deap Learning Model (이하 DL)**
   * 사용된 DL은 CNN 모델이며, variant Q-learning을 통해 학습됨
   * Input : raw pixel
   * Output : 미래 보상을 예측하는 value function
   * Result : 적용된 7개의 Atari game 중 6개의 game에서 이전 접근을 능가하였고, 6개의 game 중 3개의 game에서 인간을 능가함

## 2. Introduction
   * 오래 전부터 이미지나 음성과 같이 high-dimensional sensory inputs로부터 agent를 학습시키는 것은 RL의 오랜 과제 중 하나였으며, 이를 해결하기 위해 DL을 RL에 도입하는 연구를 진행하였다. 하지만, 도입하는 과정에서 몇가지 **문제점**이 발생하였다.
   
       1. 대부분의 DL의 Data들이 hand-labeled training data인 반면, RL의 경우 sparse, noisy, delayed한 reward signal (= scalar) 로부터 학습한다. 그러므로, DL의 경우 input에 대한 결과에 대한 계산 시간이 적은 반면, RL은 trial & error를 통해 결과를 도출하기 때문에 **delay**가 발생하므로 RL에 DL을 도입하기에 어려움이 존재한다.  <br><br>
           
       2. DL의 Data들은 서로 독립적인 반면, RL의 경우 현재 상태의 행동이 다음 상태 보상에 영향을 주는 것과 같이 **상호연관성**이 매우 높은 편이다.  <br><br>
           
       3. DL은 Data의 분포가 고정되어 있다는 가정하에 학습을 진행하는 반면, RL은 알고리즘이 **새로운 행동을 학습할 때마다 Data의 분포가 변하므로** 모순점이 발생한다.<br>
           
* 이러한 문제점은 Stochastic Gradient Descent(SGD)를 사옹한 **variant Q-learning으로 학습된 CNN 모델**을 사용함으로써 해결할 수 있었다. 추가로, correlated data와 non-stationary distributions의 문제를 완화하기 위해서 experience replay mechanism을 사용하였고, 게임에 대한 특정 정보나 데이터를 제공하지 않고 시각 데이터 및 행동, Reward, signal로만 학습하였다. 

## 3. Background
`Agent가 환경(E, Atari Emultator)와 상호작용하는 task`  
    1. 각 time-step마다 Agent는 할 수 있는 행동(aT)중에서 한가지를 선택  
    2. Action이 전달되면 Emulator는 내부 상태를 변경하고 게임 점수를 수정  
    3. Agent는 현재 화면을 나타내는 raw pixel 의 vector로 이루어진 이미지와 game score의 변화를 나타내는 reward(rt)을 전달받음  

time-step마다 Agent가 action(at) 를 선택하면 Emulator를 통해 state를 수정하고 reward(rt)가 return <br>
        
* 이때, 게임의 점수는 현재의 행동 + 이전의 행동에 의존하여 결정되고, 행동에 대한 피드백은 수많은 time-step이 진행된 후에 받게된다. 그러나, Agent는 현재의 장면만 관찰하므로 전체적인 상황을 이해하기 어렵다. 따라서, action의 sequence를 관찰하고 학습을 진행한다. 모든 sequence는 time-step에 의존되므로 MDP가 유도된다. 그러므로, sequence s_t를 시간 t의 표현으로 사용함으로써 RL을 MDP에 적용할 수 있다.  <br><br>

`MDP (Markov Decision Process)`  
    agent는 현재 화면만으로 모든 상황을 완전히 이해하는 것이 불가능하므로, action과 observatioin의 sequence에 의존하여 게임 전략을 학습한다. 

value iteration을 구할 수 있으나, 매 sequence마다 수행하기엔 비효율 적이므로 optimal Q-function을 function approximator를 이용하여 근사시킨 값을 이용한다. 이때, 근사함수는 NN을 이용한다. 
 
`Q-Network`  
다음의 Loss function을 최소화하는 방향으로 학습한다.
![image](https://user-images.githubusercontent.com/69336270/112111051-07dc6780-8bf7-11eb-9a8d-513d84e8992a.png)  <br>
이때, Gradient는 다음과 같으며, 매 time-step 마다 ρ의 확률로 s,a를 뽑아 학습하는 stochastic gradient descent을 진행하여, 최종적으로 deep learning을 이용한 Q-learning에 도달한다.
![image](https://user-images.githubusercontent.com/69336270/112111410-7de0ce80-8bf7-11eb-929f-d2052e23d4cc.png) <br>
    **[특징]**  <br>
    1. Model-Free  
      : emulator의 sample을 통해 직접 강화학습 과제 해결 <br>
    2. Off-Policy  
      : greedy strategy에 대해서 학습 후 실제 적용 시 ϵ-greedy strategy에 의해 선택

## 4. Related Work
기존의 RL 연구에서 성능이 가장 좋았던 알고리즘은 **TD-gammon**으로 backgammon 게임을 사람의 게임 실력 수준만큼 play할 수 있다. 이때, TD-gammon 역시 model-free RL 알고리즘이며, Q-learning과 유사하다.

`TD-gammon`
  - 1개의 hidden layer를 가진 multi-layer 퍼셉트론을 통해 Value Function을 근사 
  - 체스나 바둑에서는 성능이 좋지 않음

**[최근 연구]**  
`NFQ (Neural Fitted Q-learning)`  
다음의 loss function 수식을 RPROP 알고리즘을 사용하여 Q-network의 파라미터를 업데이트 및 최적화하는 알고리즘
![image](https://user-images.githubusercontent.com/69336270/112112008-4f172800-8bf8-11eb-9fae-a74fa2c24d75.png)
- 모든 데이터셋을 한번에 학습하는 batch update를 사용하므로 연산량이 크다. 

## 5. Deep Reinforcement Learning
#### 본 연구와 선행 연구의 차이점
`Experience Replay`
- agent의 experience를 각 time-step마다 저장함 (e_t = (s_t, a_t, r_t, s_t+1))
- experience samples에 Q-Learning과 mini-batch update를 적용
- experience replay를 수행한 후, agent는 greedy policy에 따라 action을 선택 및 수행
- 이러한 접근은 표준적인 online Q-learning보다 다양한 이점을 지님
    1. 각 step의 experience(e_t)는 많은 weight 업데이트에 사용될 수 있음
    2. 표본을 랜덤하게 추출함으로써, weight 업데이트의 분산을 줄게 함 
    3. 학습을 안정적으로 만듦 (behavior distribution이 평등해짐)

![image](https://user-images.githubusercontent.com/69336270/112149195-112ef980-8c22-11eb-8eeb-a143e2dff491.png)

1. 변수 초기화
  * replay memory D 초기화
  * action-value function Q 초기화 (= random weight) <br><br>
  
2. 1번째 반복문 - 에피소드 별 반복
  * sequence 초기화  : s1 = {x1}<br><br>
  
3. 2번째 반복문 - 한 에피소드 별 반복 (시간 기준)
  * ϵ의 확률로 랜덤한 action a_t를 선택
  * 이때 a_t를 선택하는 기준은 Q-value를 최대화 시키는 a_t를 선택
  * a_t를 emulator(Atari)에서 실행하고 reward r_t와 다음 이미지 x_t+1를 관찰<br><br>
  
4. Seqeunce 처리
  * sequence - s_t+1 = s_t, a_t, x_t+1 
  * sequence를 전처리 후 replay memory D에 저장<br><br>
  
5. 최적화 (**SGD**)
  * 다음과 같이 정답을 정의하고, loss를 통해 SGD로 최적화, loss를 통해 SGD로 최적화 <br>
  ![image](https://user-images.githubusercontent.com/69336270/112149711-9ca88a80-8c22-11eb-8147-aaf6096778cf.png)


## Preprocessing and Model Architecture
#### Preprocessing
![image](https://user-images.githubusercontent.com/69336270/112150193-29534880-8c23-11eb-9b23-cf4544539f17.png)
1. 원래 게임 이미지 크기는 `210x160 (pixel)`이므로, 전처리 과정을 통해 `110x84 (pixel)`로 이미지 사이즈를 줄이고 흑백으로 변환 (channel 3 to 1 - RGB > Black) 
2. CNN의 input은 정사각형이어야 하므로, 실질적으로 게임이 진행되는 부분을 제외하고 edge 자르기 : `84x84 (pixel)`

#### Modeling
[비교]
* `NN input` : sequence, action (Q function은 Q-value의 추정값을 mapping하므로 + 각 action에 대한 Q-value를 계산하기 위해 별도의 forward pass 필요)
* `DQN input` : state representation
* `DQN output` : input state에 대한 개별 action의 예측한 Q-value
즉, single forward pass를 통해 주어진 state에 대해 가능한 모든 action의 Q-value를 계산 가능 <br><br>

![image](https://user-images.githubusercontent.com/69336270/112150676-b8606080-8c23-11eb-8363-1fd8ba54f39b.png)<br>
1. last 4 frame을 stack으로 쌓기 : `84x84x4`
2. time-step마다 ϵ-greedy strategy에 의해 action을 취함
3. reward와 다음 state(St+1)를 preprocessing하여 experience을 구성 및 D에 저장
4. D에 저장된 sample을 mini-batch로 취하여 사전에 정의된 loss를 minimize하도록 GD update

## 6. Experiments
#### Testing : ATARI Game
* 7개의 유명한 ATARI game에 대해서 DQN의 성능을 테스트
* 모든 게임에 대해 동일한 조건(네트워크 구조, 학습 알고리즘, 파라미터 셋팅 등등)으로 실험 진행
* 본 실험에서 agent를 실제의 game에서 평가하기 때문에, 모든 reward는 정수로 표기함 <br><br>  
|   Reward  | mean |
|:--------:|:-----------:|
|   1  |  positive reward  | 
|   0  |    negative reward  | 
| -1 |    NOT changed  | 

#### 실험 설정 값
* 최적화 알고리즘으로 RMSProp을 사용 (minibatch size : 32)
* 학습에는 총 천만 프레임을 사용했으며, 최근 백만 프레임만을 replay memory에 사용
* 처음의 백만 프레임에서 ε값을 1에서 0.1로 선형적으로 조정 & 그 이후 프레임에서는 모두 0.1로 고정 (ε-greedy)

#### Frame Skipping technique
* agent는 모든 프레임이 아닌, k번째 프레임만을 통해 action을 선택
* 이외 프레임 - 가장 최근의 action 선택
* 본 실험에서는 4배수 프레임에서만 action 선택
* 단, Space Invader 게임에서만 k=3으로 설정 (4배수 프레임에서 비행기에서 쏘는 레이저가 사라지기 때문)
    
#### Training & Stability
`에피소드별 평균 reward 그래프 (좌 : BreakOut, 우 : Sequest)`
![image](https://user-images.githubusercontent.com/69336270/112153156-59e8b180-8c26-11eb-8401-af61d2f93c6e.png)

`평균 action value (Q) 그래프`
![image](https://user-images.githubusercontent.com/69336270/112153165-5c4b0b80-8c26-11eb-8a66-d2c70a4f71dc.png)

* 학습이 진행될 수록 Reward와 Action value 증가

#### Visualizing the Value Function
`value function 시각화 (좌 : A, 중앙 : B, 우 : C)`
![image](https://user-images.githubusercontent.com/69336270/112153448-9c11f300-8c26-11eb-85a7-b7afbca8fb04.png)
(A) enemy가 등장하면 죽였을 때 reward를 받으므로 Q값 증가
(B) 미사일을 쏘면 enemy가 죽을 가능성이 높으므로 enemey를 처치할때 받는 reward로 인하여 Q값 증가
(C) enemy가 처치되면 enemy 처치로 인한 reward가 없어지므로 Q값 감소

#### Main Evaluation
![image](https://user-images.githubusercontent.com/69336270/112153342-80a6e800-8c26-11eb-81fa-b33f76fe7f37.png)
* 적용된 7개의 Atari game 중 6개의 game에서 이전 접근을 능가하였고, 6개의 game 중 3개의 game(Breakout, Enduro, Pong)에서 인간을 능가하였다.

## 7. Conclusion
- experience replay memory로 mini-batch 방식을 사용하는 변형된 online Q-learning을 제안
- 아키텍쳐나 하이퍼 파라미터의 조정 없이, 7개의 아타리 게임에서 모두 최고의 성능을 발휘

## 8. 사후 연구
2015년 Nature 논문에서는 총 49개의 게임에 대한 결과를 도출하였다. DQN은 다른 알고리즘보다 43개 게임에서 점수가 높았고, 29개 게임에서 사람이 획득한 점수의 75% 이상을 얻음
![image](https://user-images.githubusercontent.com/69336270/112154286-75a08780-8c27-11eb-80d6-4208519b0a55.png)

#### Reference
[1] https://wordbe.tistory.com/entry/RL-%EA%B0%95%ED%99%94%ED%95%99%EC%8A%B5-part1-policy-value-function  
[2] https://ropiens.tistory.com/75  
[3] https://sumniya.tistory.com/18  
[4] https://velog.io/@kth811/1.-Playing-Atari-with-Deep-Reinforcement-Learning  
[5] https://dongminlee.tistory.com/3